# Data Acquisition Notebook

In [10]:
# importing potentially usedful libraries 
import pandas as pd
import requests
from bs4 import BeautifulSoupd
import time
import random



### exploring useful tables from FBref

In [11]:
# pulling the schedule for Real Madrid from FB ref
df =pd.read_html('https://fbref.com/en/squads/53a2f082/2024-2025/matchlogs/all_comps/schedule/Real-Madrid-Scores-and-Fixtures-All-Competitions',
                attrs = {"id":"matchlogs_for"})[0]
df = df[df['Comp'].isin(['La Liga', 'Champions Lg'])]
df.head()

df.to_clipboard(index=False)

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Opp Formation,Referee,Match Report,Notes
0,2024-08-14,21:00,Super Cup,UEFA Super Cup,Wed,Home,W,2,0,it Atalanta,NaN,NaN,52,56042,Dani Carvajal,4-1-4-1,3-4-1-2,Sandro Schärer,Match Report,NaN
1,2024-08-18,21:30,La Liga,Matchweek 1,Sun,Away,D,1,1,Mallorca,0.6,1.0,66,23010,Dani Carvajal,4-3-3,4-3-3,César Soto,Match Report,NaN
2,2024-08-25,17:00,La Liga,Matchweek 2,Sun,Home,W,3,0,Valladolid,2.1,0.4,63,70178,Dani Carvajal,4-2-3-1,4-1-4-1,Víctor García,Match Report,NaN
3,2024-08-29,20:30,La Liga,Matchweek 3,Thu,Away,D,1,1,Las Palmas,2.5,0.8,57,31192,Luka Modrić,4-2-3-1,4-1-4-1,Mateo Busquets,Match Report,NaN
4,2024-09-01,21:30,La Liga,Matchweek 4,Sun,Home,W,2,0,Betis,2.4,0.6,60,73072,Dani Carvajal,4-3-3,4-2-3-1,Javier Alberola,Match Report,NaN


In [12]:
# discovering how to pull summary stats for a certain match
df_match1_stats = pd.read_html('https://fbref.com/en/matches/c6b7a6e0/Mallorca-Real-Madrid-August-18-2024-La-Liga',
                attrs = {"id":"stats_53a2f082_summary"})[0]

df_match1_stats.head()

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0 Unnamed: 3_level_0  \
              Player                  #             Nation                Pos   
0      Kylian Mbappé                9.0             fr FRA                 FW   
1    Vinicius Júnior                7.0             br BRA                 LW   
2        Brahim Díaz               21.0             ma MAR                 LW   
3            Rodrygo               11.0             br BRA                 RW   
4    Jude Bellingham                5.0            eng ENG                 LM   

  Unnamed: 4_level_0 Unnamed: 5_level_0 Performance               ... SCA      \
                 Age                Min         Gls Ast PK PKatt  ... SCA GCA   
0             25-242                 90           0   0  0     0  ...   0   0   
1             24-037                 87           0   1  0     0  ...   7   1   
2             25-015                  3           0   0  0     0  ...   1   0   
3             23-222                 90           1   0  0     0  ...   3   0   
4             21-050                 87           0   0  0     0  ...   4   0   

  Passes                 Carries      Take-Ons       
     Cmp Att   Cmp% PrgP Carries PrgC      Att Succ  
0     25  32   78.1    5      26    5        5    3  
1     33  44   75.0    5      41    4        8    3  
2      5   5  100.0    2       5    0        0    0  
3     60  69   87.0    4      50    5        4    2  
4     63  73   86.3   14      57    2        1    1  

[5 rows x 31 columns]

In [13]:
# discovering how to pull defense specific stats for a certain match 
df_match1_defense_stats = df_match1_stats = pd.read_html('https://fbref.com/en/matches/c6b7a6e0/Mallorca-Real-Madrid-August-18-2024-La-Liga',
                attrs = {"id":"stats_53a2f082_defense"})[0]
df_match1_defense_stats.head()

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0 Unnamed: 3_level_0  \
              Player                  #             Nation                Pos   
0      Kylian Mbappé                9.0             fr FRA                 FW   
1    Vinicius Júnior                7.0             br BRA                 LW   
2        Brahim Díaz               21.0             ma MAR                 LW   
3            Rodrygo               11.0             br BRA                 RW   
4    Jude Bellingham                5.0            eng ENG                 LM   

  Unnamed: 4_level_0 Unnamed: 5_level_0 Tackles                       ...  \
                 Age                Min     Tkl TklW Def 3rd Mid 3rd  ...   
0             25-242                 90       0    0       0       0  ...   
1             24-037                 87       0    0       0       0  ...   
2             25-015                  3       0    0       0       0  ...   
3             23-222                 90       1    1       1       0  ...   
4             21-050                 87       6    5       5       1  ...   

  Challenges             Blocks         Unnamed: 18_level_0  \
         Att   Tkl% Lost Blocks Sh Pass                 Int   
0          0    NaN    0      0  0    0                   0   
1          0    NaN    0      0  0    0                   0   
2          0    NaN    0      0  0    0                   0   
3          1  100.0    0      2  0    2                   0   
4          3  100.0    0      2  0    2                   0   

  Unnamed: 19_level_0 Unnamed: 20_level_0 Unnamed: 21_level_0  
              Tkl+Int                 Clr                 Err  
0                   0                   0                   0  
1                   0                   0                   0  
2                   0                   0                   0  
3                   1                   0                   0  
4                   6                   0                   0  

[5 rows x 22 columns]

## Functions for Repetitive Pulling 

In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random

BASE_URL = "https://fbref.com"

HEADERS_LIST = [
    {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"},
    {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7)"},
    {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64)"},
    {"User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 14_0 like Mac OS X)"}
]

def fetch_match_report_links(main_url, headers):
    print("⏳ Waiting politely before fetching match links...")
    time.sleep(random.uniform(10, 15))

    for attempt in range(3):
        current_headers = random.choice(HEADERS_LIST)
        response = requests.get(main_url, headers=current_headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")
            table = soup.find("table", {"id": "matchlogs_for"})
            links = []
            for row in table.find_all("tr"):
                link_tag = row.find("a", text="Match Report")
                if link_tag:
                    links.append(BASE_URL + link_tag["href"])
            return links
        else:
            print(f"⚠️ Failed to fetch match list (attempt {attempt+1}), status code {response.status_code}")
            time.sleep(random.uniform(10, 15))

    print("❌ Failed to fetch match report links after retries.")
    return []

def scrape_player_stats(match_url, headers):
    print("⏳ Waiting politely before requesting page...")
    time.sleep(random.uniform(10, 15))

    current_headers = random.choice(HEADERS_LIST)
    response = requests.get(match_url, headers=current_headers)
    if response.status_code != 200:
        print(f"❌ Failed to load {match_url}")
        return None

    soup = BeautifulSoup(response.content, "html.parser")

    # Extract opponent and date
    title_tag = soup.find("h1")
    title_text = title_tag.text.strip() if title_tag else ""
    opponent = "Unknown"
    if "Real Madrid" in title_text:
        parts = title_text.split(" vs ")
        if len(parts) == 2:
            opponent = parts[1] if parts[0].strip().startswith("Real Madrid") else parts[0]

    date_tag = soup.select_one(".scorebox_meta time")
    match_date = date_tag['datetime'] if date_tag and 'datetime' in date_tag.attrs else "Unknown"

    # Extract all relevant tables
    def scrape_table(table_id):
        table = soup.find("table", {"id": table_id})
        if table:
            try:
                df = pd.read_html(str(table), header=[0, 1])[0]
                df.columns = [' '.join(col).strip() if 'Unnamed' not in col[0] else col[1] for col in df.columns]
                df = df[df['Player'].notna()]
                df = df[~df['Player'].str.contains(r'\d+\s+Players', na=False)]
                return df
            except Exception as e:
                print(f"⚠️ Error parsing table '{table_id}' from {match_url}: {e}")
                return None
        else:
            print(f"⚠️ Table '{table_id}' not found in {match_url}")
            return None

    summary_df = scrape_table("stats_53a2f082_summary")
    defense_df = scrape_table("stats_53a2f082_defense")
    passing_df = scrape_table("stats_53a2f082_passing")

    # Merge all available tables on "Player"
    dfs = [df for df in [summary_df, defense_df, passing_df] if df is not None]

    if not dfs:
        return None

    merged_df = dfs[0]
    for df in dfs[1:]:
        df = df.drop(columns=[col for col in df.columns if col in merged_df.columns and col != "Player"])
        merged_df = pd.merge(merged_df, df, on="Player", how="outer")

    merged_df["Match URL"] = match_url
    merged_df["Opponent"] = opponent
    merged_df["Date"] = match_date
    return merged_df

def scrape_all_matches(main_url, headers):
    match_links = fetch_match_report_links(main_url, headers)
    all_stats = []

    for match_url in match_links:
        print(f"Scraping: {match_url}")
        stats_df = scrape_player_stats(match_url, headers)
        if stats_df is not None:
            all_stats.append(stats_df)

    if all_stats:
        return pd.concat(all_stats, ignore_index=True)
    else:
        print("❌ No data scraped.")
        return pd.DataFrame()

# Run it - kachow
url = "https://fbref.com/en/squads/53a2f082/2024-2025/matchlogs/all_comps/schedule/Real-Madrid-Scores-and-Fixtures-All-Competitions"
headers = {"User-Agent": "Mozilla/5.0"}

player_stats_df = scrape_all_matches(url, headers)



⏳ Waiting politely before fetching match links...


/var/folders/_4/7497cf992_zb43qwn6j7xrgm0000gn/T/ipykernel_33190/3659086326.py:28: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  link_tag = row.find("a", text="Match Report")


Scraping: https://fbref.com/en/matches/14cdbbf5/Real-Madrid-Atalanta-August-14-2024-UEFA-Super-Cup
⏳ Waiting politely before requesting page...
⚠️ Table 'stats_53a2f082_defense' not found in https://fbref.com/en/matches/14cdbbf5/Real-Madrid-Atalanta-August-14-2024-UEFA-Super-Cup
⚠️ Table 'stats_53a2f082_passing' not found in https://fbref.com/en/matches/14cdbbf5/Real-Madrid-Atalanta-August-14-2024-UEFA-Super-Cup
Scraping: https://fbref.com/en/matches/c6b7a6e0/Mallorca-Real-Madrid-August-18-2024-La-Liga
⏳ Waiting politely before requesting page...
Scraping: https://fbref.com/en/matches/3f2f6c96/Real-Madrid-Valladolid-August-25-2024-La-Liga
⏳ Waiting politely before requesting page...
Scraping: https://fbref.com/en/matches/173a87bb/Las-Palmas-Real-Madrid-August-29-2024-La-Liga
⏳ Waiting politely before requesting page...
Scraping: https://fbref.com/en/matches/f8b3c447/Real-Madrid-Real-Betis-September-1-2024-La-Liga
⏳ Waiting politely before requesting page...
Scraping: https://fbref.com/

In [16]:
#player_stats = player_stats_df.copy()

player_stats.head(5)


,Player,#,Nation,Pos,Age,Min,Performance Gls,Performance Ast,Performance PK,Performance PKatt,...,Long Att,Long Cmp%,Ast,xAG,xA,KP,1/3,PPA,CrsPA,PrgP
0,Kylian Mbappé,9.0,fr FRA,FW,25-238,82,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Brahim Díaz,21.0,ma MAR,"FW,MF",25-011,8,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Vinicius Júnior,7.0,br BRA,FW,24-033,87,0,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arda Güler,15.0,tr TUR,MF,19-171,3,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Rodrygo,11.0,br BRA,FW,23-218,75,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Brief Formatting of DF

In [33]:
# remove matches not in Champions League or La Liga 
unwanted_keywords = ["UEFA-Super-Cup", "Copa-del-Rey", "Supercopa-de-Espana"]
player_stats_df = player_stats_df[~player_stats_df['Match URL'].str.contains('|'.join(unwanted_keywords), na=False)]

# clean up df
player_stats_df.columns = player_stats_df.columns.str.replace("Performance", "", regex=False)
player_stats_df['#'] = player_stats_df['#'].astype(int)
columns_to_remove = [' Fls', ' Fld', ' Off', ' Crs', ' TklW', ' OG', ' PKwon', ' PKcon']
player_stats_df = player_stats_df.drop(columns=columns_to_remove, errors='ignore')


# provide a list of coolumn names is player_stats_df
player_stats_df.columns.tolist()


player_stats_df.head()





,Date,Opponent,Player,#,Nation,Pos,Age,Min,Gls,Ast,...,Long Att,Long Cmp%,Ast,xAG,xA,KP,1/3,PPA,CrsPA,PrgP
16,2024-08-18,Mallorca,Kylian Mbappé,9,fr FRA,FW,25-242,90,0,0,...,1.0,100.0,0.0,0.0,0.1,0.0,3.0,0.0,0.0,5.0
17,2024-08-18,Mallorca,Vinicius Júnior,7,br BRA,LW,24-037,87,0,1,...,2.0,100.0,1.0,0.3,0.4,6.0,0.0,3.0,0.0,5.0
18,2024-08-18,Mallorca,Brahim Díaz,21,ma MAR,LW,25-015,3,0,0,...,0.0,NaN,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0
19,2024-08-18,Mallorca,Rodrygo,11,br BRA,RW,23-222,90,1,0,...,6.0,16.7,0.0,0.0,0.1,1.0,5.0,1.0,0.0,4.0
20,2024-08-18,Mallorca,Jude Bellingham,5,eng ENG,LM,21-050,87,0,0,...,8.0,62.5,0.0,0.1,0.2,2.0,8.0,3.0,0.0,14.0


# Imputation

### Date and Opponent Imputation

In [31]:
import pandas as pd
import re

# Month names to avoid capturing as part of team name
months = {
    "january", "february", "march", "april", "may", "june",
    "july", "august", "september", "october", "november", "december"
}

noise_words = {"el", "derbi", "madrileno"}

def extract_opponent(url):
    try:
        slug = url.split("/matches/")[1].split("/")[1].lower()
        parts = slug.split("-")

        for i in range(len(parts) - 1):
            if parts[i] == "real" and parts[i + 1] == "madrid":
                # Opponent before Real Madrid
                before = parts[max(0, i - 3):i]
                before_clean = [p for p in before if p not in noise_words and p not in months]
                if before_clean:
                    return " ".join(before_clean).title()

                # Opponent after Real Madrid
                after = parts[i + 2:i + 5]
                after_clean = []
                for p in after:
                    if p in months:
                        break
                    if p not in noise_words:
                        after_clean.append(p)
                if after_clean:
                    return " ".join(after_clean).title()
        return None
    except:
        return None

def extract_date(url):
    try:
        match = re.search(r'(January|February|March|April|May|June|July|August|September|October|November|December)-\d{1,2}-\d{4}', url)
        if match:
            parsed = pd.to_datetime(match.group(0), format='%B-%d-%Y')
            return parsed.strftime('%Y-%m-%d')
        return None
    except:
        return None

# Apply to DataFrame
player_stats_df['Opponent'] = player_stats_df['Match URL'].apply(extract_opponent)
player_stats_df['Date'] = player_stats_df['Match URL'].apply(extract_date)


### Impute Competition Name

In [38]:
import pandas as pd
import re

def extract_competition(url):
    try:
        # Competition appears after the date, just before the league suffix
        # e.g. "...-November-5-2024-Champions-League"
        match = re.search(r'-\d{4}-([A-Za-z\-]+)$', url.split('/', 5)[-1])
        if match:
            return match.group(1).replace('-', ' ')
        return None
    except:
        return None

# Apply to DataFrame
player_stats_df['Competition'] = player_stats_df['Match URL'].apply(extract_competition)


In [ ]:
# Reorder columns
cols = player_stats_df.columns.tolist()
front = ['Date','Competition', 'Opponent', 'Player']

# Keep the rest in original order, excluding the ones already moved
rest = [col for col in cols if col not in front]
# Apply new order
player_stats_df = player_stats_df[front + rest]

### Finalize Data

In [ ]:
from IPython.display import display, HTML

# View full database to ensure it looks good
display(HTML(player_stats_df.to_html(max_rows=1000, max_cols=1000)))

In [39]:
# saved to clipboard for easy pasting into Excel, then save as csv due to issues converting to csv through python
player_stats_df.to_clipboard(index=False)



NameError: name 'player_stats_df' is not defined